In [30]:
# %matplotlib osx 
# ^ UNCOMMENT THIS LINE IF USING MAC

%matplotlib qt 
# ^ UNCOMMENT THIS LINE IF USING WINDOWS

#Utility
import numpy as np
import scipy.integrate
import glob #to get directories
import re

#Plotting & animation
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

#Astro
import astropy.constants as const
import astropy.units as u

#GUI
from __future__ import print_function
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import GridspecLayout

## Physics setup

In [2]:
'''
    Represents the differential equation for the law of gravitation.
    rv: an array containing the positions and velocities of 2 objects
    t: time
'''
def gravitation(rv, t, mass_star, mass_planet):
    r1 = rv[:2] 
    v1 = rv[2:4] 
    r12 = np.linalg.norm(r1)
    
    dv1bydt = mass_planet * (-r1)/(pow(r12, 3))
    dr1bydt = v1
    
    derivatives = np.concatenate((dr1bydt, dv1bydt))
    return derivatives

## Classes setup

In [3]:
class AstroObject:
    
    def __init__(self, mass, init_position, init_velocity, radius):
        self.mass = mass
        self.init_position = init_position
        self.init_velocity = init_velocity
        self.radius = radius
    
    def set_mass(self, mass):
        self.mass = mass
    
    def set_distance(self, init_position):
        self.init_position = init_position

In [4]:
class Planet(AstroObject):
    
    '''
    mass: mass of planet in Earth masses 
    init_position: planet's distance from star (au) [distance_x, distance_y]
    init_velocity: planet's velocity (m/s) [vx, vy]
    color: for displaying orbit in the animation
    image: icon to use for planet in the animation
    implemented: whether to show this planet in the animation
    '''
    
    def __init__(self, mass, init_position, init_velocity, color, image, implemented=True):
        mass_planet = mass * 10**(-5)
        super().__init__(mass_planet, init_position, init_velocity, ((mass_planet/7.36*np.pi)**(1./3.)))
        self.implemented = implemented
        self.color = color
        self.image = image
        
    def __str__(self):
        return f'Planet: Mass={self.mass}, Radius={self.radius}, Distance={self.init_position[0]}, Velocity={self.init_velocity[1]}'
    
    def set_implemented(self, status):
        self.implemented = status
    

In [5]:
class Star(AstroObject):
    
    '''
    mass: mass of star in solar masses
    radius: radius of star in Earth radii (sun is ~109 Earth radii)
    init_position: list representing x, y position of star (defaults to origin)
    init_velocity: vector representing x and y components of star's velocity (defaults to 0; ie. stationary)
    '''
    def __init__(self, mass, radius, init_position = [0,0], init_velocity = [0, 0]):
#         self.mass = mass 
#         self.position = position 
#         self.velocity = velocity
#         self.radius = radius
        super().__init__(mass, init_position, init_velocity, radius)
    
        
    def __str__(self):
        return f'Star {self.name}. Mass={self.mass}, Radius={self.radius}'   

In [6]:
class SolarSystem: 
    
    def __init__(self, star, planets=[]):
        self.star = star
        self.planets = planets
    
    def get_planet_masses(self):
        masses = []
        for planet in self.planets:
            if planet.implemented:
                masses.append(planet.mass)
        return masses
    
    def get_planet_distances(self):
        distances = []
        for planet in self.planets:
            if planet.implemented:
                distances.append(planet.init_position[0])
        return distances
    
    def most_massive_planet(self):
        masses = self.get_planet_masses()
        largest = max(masses)
        for planet in self.planets:
            if planet.implemented and planet.mass == largest:
                return planet
        return None
    
    def farthest_planet(self):
        distances = self.get_planet_distances()
        farthest = max(distances)
        for planet in self.planets:
            if planet.implemented and planet.init_position[0] == farthest:
                return planet
        return None
            

## Setting planet properties

In [7]:
planet_1 = widgets.Checkbox(
                value=False,
                description='Planet 1',
                disabled=False,
                continous_update=True,
                indent=False
                )

planet_2 = widgets.Checkbox(
                value=False,
                description='Planet 2',
                disabled=False,
                continous_update=True,
                indent=False
                )

planet_3 = widgets.Checkbox(
                value=False,
                description='Planet 3',
                disabled=False,
                continous_update=True,
                indent=False
                )

planet_4 = widgets.Checkbox(
                value=False,
                description='Planet 4',
                disabled=False,
                continous_update=True,
                indent=False
                )

planet_5 = widgets.Checkbox(
                value=False,
                description='Planet 5',
                disabled=False,
                continous_update=True,
                indent=False
                )

mass_1 = widgets.FloatSlider(
            value=0.5,
            min=0.5,
            max=50.0,
            step=0.5,
            description='Mass:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )

velocity_1 = widgets.FloatSlider(
            min=0.1,
            max=1.0,
            step=0.05,
            description='Velocity:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            )

distance_1 = widgets.FloatSlider(
            value=1,
            min=0.5,
            max=10.0,
            step=0.1,
            description='Distance From The Sun:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )

mass_2 = widgets.FloatSlider(
            value=0.5,
            min=0.5,
            max=50.0,
            step=0.5,
            description='Mass:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )

velocity_2 = widgets.FloatSlider(
            min=0.1,
            max=1.0,
            step=0.05,
            description='Velocity:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            )

distance_2 = widgets.FloatSlider(
            value=1,
            min=0.5,
            max=10.0,
            step=0.1,
            description='Distance From The Sun:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )
mass_3 = widgets.FloatSlider(
            value=0.5,
            min=0.5,
            max=50.0,
            step=0.5,
            description='Mass:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )

velocity_3 = widgets.FloatSlider(
            min=0.1,
            max=1.0,
            step=0.05,
            description='Velocity:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            )

distance_3 = widgets.FloatSlider(
            value=1,
            min=0.5,
            max=10.0,
            step=0.1,
            description='Distance From The Sun:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )
mass_4 = widgets.FloatSlider(
            value=0.5,
            min=0.5,
            max=50.0,
            step=0.5,
            description='Mass:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )

velocity_4 = widgets.FloatSlider(
            min=0.1,
            max=1.0,
            step=0.05,
            description='Velocity:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            )

distance_4 = widgets.FloatSlider(
            value=1,
            min=0.5,
            max=10.0,
            step=0.1,
            description='Distance From The Sun:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )
mass_5 = widgets.FloatSlider(
            value=0.5,
            min=0.5,
            max=50.0,
            step=0.5,
            description='Mass:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )

velocity_5 = widgets.FloatSlider(
            min=0.1,
            max=1.0,
            step=0.05,
            description='Velocity:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            )

distance_5 = widgets.FloatSlider(
            value=1,
            min=0.5,
            max=10.0,
            step=0.1,
            description='Distance From The Sun:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            )

In [8]:
grid = GridspecLayout(5, 4, height='300px')
grid[0,0] = planet_1
grid[1,0] = planet_2
grid[2,0] = planet_3
grid[3,0] = planet_4
grid[4,0] = planet_5
grid[0,1] = mass_1
grid[1,1] = mass_2
grid[2,1] = mass_3
grid[3,1] = mass_4
grid[4,1] = mass_5
grid[0,2] = velocity_1
grid[1,2] = velocity_2
grid[2,2] = velocity_3
grid[3,2] = velocity_4
grid[4,2] = velocity_5
grid[0,3] = distance_1
grid[1,3] = distance_2
grid[2,3] = distance_3
grid[3,3] = distance_4
grid[4,3] = distance_5

In [9]:
def f(planet_1, planet_2, planet_3, planet_4, planet_5):
    print((planet_1, planet_2, planet_3, planet_4, planet_5))

out = widgets.interactive_output(f, {'planet_1': planet_1, 'planet_2': planet_2, 'planet_3': planet_3, 'planet_4': planet_4, 'planet_5': planet_5})

display(grid, out)

GridspecLayout(children=(Checkbox(value=False, description='Planet 1', indent=False, layout=Layout(grid_area='…

Output()

In [42]:
# Creating the solar system with values from GUI sliders

p1 = Planet(mass_1.value, [distance_1.value, 0], [0, velocity_1.value], color='yellow', image = 'venus.png', implemented=planet_1.value)
p2 = Planet(mass_2.value, [distance_2.value, 0], [0, velocity_2.value], color='aquamarine', image = 'earth.png', implemented=planet_2.value)
p3 = Planet(mass_3.value, [distance_3.value, 0], [0, velocity_3.value], color='red', image = 'mars.png', implemented=planet_3.value)
p4 = Planet(mass_4.value, [distance_4.value, 0], [0, velocity_4.value], color='orange', image = 'jupiter.png', implemented=planet_4.value)
p5 = Planet(mass_5.value, [distance_5.value, 0], [0, velocity_5.value], color='cyan', image = 'neptune.png', implemented=planet_5.value)

sun = Star(mass = 1, radius = 109.0)
test_system = SolarSystem(sun, [p1, p2, p3, p4, p5])

## Simulation Setup

In [41]:
# Initialize writer 
metadata = dict(title='Orbit Test', artist='Matplotlib')
writer = FFMpegWriter(fps=50, metadata=metadata, bitrate=200000) # change fps for different frame rates
fig = plt.figure(dpi=200)

In [46]:
# Set up & solve ODE for motion of each planet in the solar system
# Automatically set timespan to run the simulation, so that we get at least 1 full orbit of each planet
# Sets plot dimensions automatically for simulation display

all_orbit_solutions = []
# Sets time needed to run simulation according to how far the farthest planet is (rough estimate only)
time_span = np.linspace(0, max(test_system.get_planet_distances()) * 300, 5000) 
axes_limits_x = [-max(test_system.get_planet_distances()) * 10, max(test_system.get_planet_distances()) * 10]
axes_limits_y = [-max(test_system.get_planet_distances()) * 10, max(test_system.get_planet_distances()) * 5]
lightcurve = []
time_dim = []


for planet in test_system.planets:
    if planet.implemented:
        init_params = np.array([planet.init_position, planet.init_velocity])
        init_params = init_params.flatten()
        
        sol = scipy.integrate.odeint(gravitation, init_params, time_span, args=(planet.mass, test_system.star.mass))
        sol_for_planet = sol[:, :2]
        all_orbit_solutions.append(sol_for_planet)
        
        
largest_rad = max(test_system.planets[i].radius for i in range(len(test_system.planets)))
largest_depth = largest_rad/test_system.star.radius
depth_transit = 1 - (largest_depth * 10)

## Animation

In [47]:
# Automatically set up directories

all_videos = glob.glob("videos_generated\*")
all_digits = []
for vid in all_videos:
    matched = re.findall(r'\d+.', vid)
    digits = int(re.findall(r'\d+', matched[0])[0])
    all_digits.append(digits)
next_idx = max(all_digits) + 1

In [48]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,8), gridspec_kw={'height_ratios': [3, 1]})
bg_img = mpimg.imread("space.jpeg")


# Setting up image icons for planets
def getImage(path):
    return OffsetImage(plt.imread(path, format="png"), zoom=.01)

paths = []
for planet in test_system.planets:
    paths.append(planet.image)

with writer.saving(fig, "videos_generated\orbit_test_" + str(next_idx) + ".mp4", dpi=200):
    
    for i in range(len(time_span)):

        # Animation of orbiting planets
        
        ax1.clear()
        
        for planet_idx in range(len(test_system.planets)):
            if test_system.planets[planet_idx].implemented:
                planet_sols = all_orbit_solutions[planet_idx]
                
                planet_img = mpimg.imread(test_system.planets[planet_idx].image)
                
                ax1.plot(planet_sols[:i, 0], planet_sols[:i, 1], color = test_system.planets[planet_idx].color, alpha=0.5) # path

                
                ab = AnnotationBbox(getImage(paths[planet_idx]), (planet_sols[i,0], planet_sols[i,1]), frameon=False)
                ax1.add_artist(ab)
                
        ax1.imshow(bg_img, extent = [axes_limits_x[0], axes_limits_x[1], axes_limits_y[0], axes_limits_y[1]]) # Uncomment to work on background image
        ax1.scatter(0, 0, color="orange",marker="*", s=50, zorder=5) # star
        
        ax1.set_xlim(axes_limits_x[0], axes_limits_x[1])
        ax1.set_ylim(axes_limits_y[0], axes_limits_y[1])
        ax1.set_title('Solar System Animation')


        # Animation of lightcurve with transits
        
        ax2.clear()
        
        # 1. Define observation of transit to be when planet moves back to where it started (position = [orig_distance, 0]) (+/- a bit)
        # 2. Observed magnitude is 1.0 if nothing is transiting
        # 3. Observed magnitude is 1.0 - (planet size/star ratio)^2
        
        flux_level = 1.0
        for planet_idx in range(len(test_system.planets)):
            if test_system.planets[planet_idx].implemented:
                planet_sols = all_orbit_solutions[planet_idx]
                dist = test_system.planets[planet_idx].init_position
                loc = [planet_sols[i, 0], planet_sols[i, 1]]
                if abs(loc[0] - dist[0]) < 0.1 and abs(loc[1]) < 0.1:
                # Transit depth = (radius of planet/radius of star)^2. However, only using pure ratio to keep it visible here.
                    flux_level -= (test_system.planets[planet_idx].radius/test_system.star.radius)
        
        lightcurve.append(flux_level)
        time_dim.append(i)
            
        ax2.scatter(i, flux_level, color = 'orange', marker = 'o', s=10, zorder = 5)
        ax2.plot(time_dim, lightcurve, color='red', alpha=0.3)
        
        
        ax2.set_xlim(0, len(time_span))
        ax2.set_ylim(depth_transit, 1.002)
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Flux')
        ax2.set_title('Lightcurve')
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()

## Playback generated video

In [49]:
all_digits.append(next_idx)
all_digits.sort()

def solar_sim(Simulations):
    display(HTML("""<video width="800" height="600" controls><source src="videos_generated/orbit_test_{}.mp4" type="video/mp4"></video>""".format(Simulations)))

widgets.interact(solar_sim, Simulations = all_digits);

interactive(children=(Dropdown(description='Simulations', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…